In [53]:
"""
    정리
    1. agent를 통해서 policy(랜덤값이 앱실론보다 작으면 랜덤 인덱스와 랜덤 action을, 
                            그렇지 않으면 내부 신경망(LSTM)에 state값을 넣어 
                            가능한 모든 action에 대한 action_values(Q_values) 중 가장 큰 값에 해당하는 인덱스와 그 값을 반환한다.
                            이때 사용되는 내부 신경망은 main_network이다.)
        에 맞는 action_index와 action_value를 찾아낸다.
    2. 찾아낸 action_index와 action_value를 env에 넣어서 RF 모델을 돌린 뒤 next_state(다음 action 집합)와 reward(정확도)를 찾아낸다.
    3. memory(replay buffer)에 현재 state와 action_index, 그리고 위에서 도출된 next_state, reward를 넣는다.
    4. replay buffer에서 원하는 batch size만큼 sample(state, action_index, reward, next_state)을 뽑아낸다.
    5. sample의 state를 main_network에 넣어서 도출된 값들 중 action index에 맞는 값을 뽑고 이를 main Q 값으로 한다.
    6. sample의 next_state를 target_network에 넣어서 값을 도출한 뒤, 그 중 가장 큰 값을 찾아낸다.
    7. 위에서 찾아낸 값을 Q라고 한다면, reward + gamma(discount factor)*Q = target Q로 정한다.
    8. mainQ와 targetQ를 loss function에 넣어서 내부 신경망 파라미터를 역전파로 업데이트 한다. 이때 옵티마이저도 사용된다.
    9. 이 과정을 반복하여 main Q가 target Q에 가까워질 수 있게 한다.
"""

"""
    할것
    1. state 설정 -> 현재 상태의 모든 가능한 하이퍼파라미터 조합 space
    2. state에서 action 도출 과정 표현
    3. replay memory 설정
    
    고민
    1. multi agent를 사용해야하나?
    
    state 설정
    만약 넣은 state가 100,None,2,1,0,0(defalut) 라면, action value는 
    3^6=729개의 조합에 대해 q-value를 뽑고 그 중 하나를 고르는 것.
    따라서 state는 (729, 5)의 크기를 갖는다.
    
    deep q 이유
    만약 일반 q learning을 쓸 때, 
    하나의 하이퍼파라미터 당 10개의 tracking을 한다고 가정하면, 10^6만큼의 q-table이 필요하게 된다.
    따라서 시/공간 효율성을 위해 하이퍼파라미터 조합에 대한 q-value 값을 표현해주는 신경망을 학습시키는 것이 도움이 될 수 있다.
    
    우리는 하나의 state당 729가지의 action이 생기고, 이러한 state
    각 하이퍼파라미터 당 10번의 tracking을 한다고 가정하면 ?? 얼만큼의 메모리가 소요된다고 할 수 있는가?
"""



'\n    할것\n    1. state 설정 -> 현재 상태의 모든 가능한 하이퍼파라미터 조합 space\n    2. state에서 action 도출 과정 표현\n    3. replay memory 설정\n    \n    고민\n    1. multi agent를 사용해야하나?\n    \n    state 설정\n    만약 넣은 state가 100,None,2,1,0,0(defalut) 라면, action value는 \n    3^6=729개의 조합에 대해 q-value를 뽑고 그 중 하나를 고르는 것.\n    따라서 state는 (729, 5)의 크기를 갖는다.\n    \n    deep q 이유\n    만약 일반 q learning을 쓸 때, \n    하나의 하이퍼파라미터 당 10개의 tracking을 한다고 가정하면, 10^6만큼의 q-table이 필요하게 된다.\n    따라서 시/공간 효율성을 위해 하이퍼파라미터 조합에 대한 q-value 값을 표현해주는 신경망을 학습시키는 것이 도움이 될 수 있다.\n    \n    우리는 하나의 state당 729가지의 action이 생기고, 이러한 state\n    각 하이퍼파라미터 당 10번의 tracking을 한다고 가정하면 ?? 얼만큼의 메모리가 소요된다고 할 수 있는가?\n'

In [54]:
import gym
import numpy as np
import pandas as pd
import random
from collections import deque, namedtuple
import copy
from itertools import product
import torch
from torch import nn as nn
from torch import optim
import torch.nn.functional as F

In [55]:
device = torch.device('cpu')

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

class h2y2_RF_Model():
    def __init__(self, cur_hyperparameter):
        self.model = RandomForestClassifier(**cur_hyperparameter, random_state=42)
        self.data = load_breast_cancer()
        x_data = pd.DataFrame(self.data.data, columns=self.data.feature_names)
        y_data = self.data.target
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(x_data, y_data, test_size=0.3, random_state=42)
        
        
    def evaluate(self):
        self.model.fit(self.train_x, self.train_y)
        predict = self.model.predict(self.test_x)
        return accuracy_score(self.test_y, predict)
    

In [57]:
class H2Y2_env:
    
    def __init__(self):
        # initial
        self.comb_config = [[-50,0,50], [-2,0,2], [-1,0,1], [-1,0,1], [-0.1,0,0.1], [-0.1,0,0.1]]
        self.hyperparameter_list = ['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'min_weight_fraction_leaf', 'min_impurity_decrease', 'max_samples']
    
    
    # 하이퍼파라미터의 범위를 제한해주는 함수
    def check_bound(self, comb):        
        comb_sum = comb[1] + comb[2]
        if comb[0] == 'n_estimators':
            if comb_sum > 0:
                return int(comb_sum)
        elif comb[0] == 'max_depth':
            if comb_sum > 0:
                return int(comb_sum)
        elif comb[0] == 'min_samples_leaf':
            if comb_sum > 0:
                return int(comb_sum)
        elif comb[0] == 'min_samples_split':
            if comb_sum > 1:
                return int(comb_sum)
        elif comb[0] == 'min_weight_fraction_leaf':
            if comb_sum >= 0 and comb_sum <= 0.5:
                return float(comb_sum)
        elif comb[0] == 'min_impurity_decrease':
            if comb_sum >= 0 and comb_sum <= 1:
                return float(comb_sum)
        return comb[1]
    
    def check_bound_ver2(self, comb):
        sample_state = []
        for i in range(len(comb[2])):
            comb_sum = comb[1] + comb[2][i]
            if comb[0] == 'n_estimators':
                if comb_sum > 0:
                    sample_state.append(int(comb_sum))
            elif comb[0] == 'max_depth':
                if comb_sum > 0:
                    sample_state.append(int(comb_sum))
            elif comb[0] == 'min_samples_leaf':
                if comb_sum > 0:
                    sample_state.append(int(comb_sum))
            elif comb[0] == 'min_samples_split':
                if comb_sum > 1:
                    sample_state.append(int(comb_sum))
            elif comb[0] == 'min_weight_fraction_leaf':
                if comb_sum >= 0 and comb_sum <= 0.5:
                    sample_state.append(float(comb_sum))
            elif comb[0] == 'min_impurity_decrease':
                if comb_sum >= 0 and comb_sum <= 1:
                    sample_state.append(float(comb_sum))
            else:
                sample_state.append(comb[1])
        return sample_state
    
    # hyper-parameter vector를 받아서 다음으로 가능한 모든 조합을 반환해주는 함수
    def make_state(self, cur_comb):
        comb = list(product(*self.comb_config))
        state = [tuple(map(self.check_bound, zip(self.hyperparameter_list, cur_comb, tuple_comb))) for tuple_comb in comb]
        # print(state)
        return state
        
    # q_value로 도출된 action_index가 어떤 observation을 가리키는지 확인한다.
    def mapping_action(self, state, action_index):
        all_state = list(product(*state))
        # print(len(all_state))
        return all_state[action_index]
        
    # hyper-parameter vector를 받아서 파라미터마다 가능한 값을 2차원으로 반환해주는 함수 ex)[[90, 100, 110], [2, 4, 6], [20, 25, 30], ...]
    def make_state_ver2(self, cur_comb):
        state = []
        for name, cur, comb in zip(self.hyperparameter_list, cur_comb, self.comb_config):
            state.append(self.check_bound_ver2([name,cur,comb]))
        # print(state)
        return state
    
    # env의 초기 state 설정, state는 hyper-parameter의 가능한 모든 조합으로 정의한다.
    def reset(self):
        init_hp = [random.randint(1, 100), random.randint(1, 100), random.randint(2, 100), random.randint(1, 100), random.random()/2, random.random()]
        state = self.make_state(init_hp)
        return state
        
    # action을 넣어서 next_state와 reward를 반환하는 함수
    def step(self, state, action_index):
        done = 0
        cur_comb = state[action_index]
        cur_hyperparameter = dict(zip(self.hyperparameter_list, cur_comb))
        rf_model = h2y2_RF_Model(cur_hyperparameter)
        reward = rf_model.evaluate()
        next_state = self.make_state(cur_comb)
        if reward == 1:
            done = 1
        return next_state, reward, done

    def step_ver2(self, state, action_index):
        # action을 넣어서 next_state와 reward를 반환하는 함수
        done = 0
        cur_comb = self.mapping_action(state, action_index) # state에 맞게 현재 조합 매핑
        cur_hyperparameter = dict(zip(self.hyperparameter_list, cur_comb))
        rf_model = h2y2_RF_Model(cur_hyperparameter)
        reward = rf_model.evaluate()
        next_state = self.make_state_ver2(cur_comb)
        if reward == 1:
            done = 1
        return next_state, reward, done
        

In [161]:
class H2Y2_Agent:
    def __init__(self):
        self.gamma = 0.99 # discount factor
        self.t_step = 0
        self.target_update = 4
        self.batch_size = 32
        self.action_size = 6
        # self.main_network = Network(state_size,action_size).float().to(device)
        # self.target_network = Network(state_size,action_size).float().to(device)
        self.main_network = Network(input_size=(729,6), out_size=729).to(device)
        self.target_network = copy.deepcopy(self.main_network).eval()
        self.hidden_state, self.cell_state = self.main_network.init_hidden_states(bsize=1)
        self.memory = ReplayBuffer(action_size = self.action_size, buffer_size = 10000, batch_size = self.batch_size, seed=42)
        self.optimizer = optim.Adam(self.main_network.parameters(), lr = 0.01)

    
    def select_action(self, state, eps=0.):
        # "내부의 신경망에 state를 넣어 모든 q_value를 뽑고, argmax로 선택"
        
        state = torch.from_numpy(np.array(state)).float().unsqueeze(0).to(device)
        self.main_network.eval()
        with torch.no_grad(): # 연산속도 증가
            action_values = self.main_network.forward(state, bsize=1, time_step=1, hidden_state=self.hidden_state, cell_state=self.cell_state)
            # print(action_values)
            # print(action_values.shape)
        self.main_network.train()
    
        # q_value를 최대로 만드는 action의 인덱스를 선택한다.
        if random.random() > eps:
            max_index = torch.argmax(action_values.cpu().data.numpy())
            return max_index
        else:
            random_index = random.choice(np.arange(self.action_size))
            return random_index
            
    def step(self, state, action_index, reward, next_state, done):
        # 메모리에 현재의 state, action_index, reward, next_state, done을 추가한다.
        self.memory.add(state, action_index, reward, next_state, done)
        
        # target_update로 정해둔 step마다, batch_size만큼의 샘플을 가지고 main_network를 학습시킨다.
        self.t_step = (self.t_step + 1) % self.target_update
        if self.t_step == 0:
            
            # batch_size만큼의 sample이 memory에 있으면 학습을 실행한다.
            if len(self.memory) > self.batch_size:
                experiences = self.memory.sample()
                self.learn(experiences, self.gamma)
    
    # main_network의 파라미터를 학습하는 함수
    def learn(self, experiences, gamma):
        states, actions, reward, next_states, dones = experiences
        hidden_batch, cell_batch = self.main_network.init_hidden_states(bsize=self.batch_size)

        # target_network를 통해 next_state에 대한 q_value 값을 도출하고, 그 중 max인 값을 선택한다.
        Q_targets_next, _ = self.target_network.forward(next_states,bsize=self.batch_size, time_step=1, hidden_state=hidden_batch, cell_state=cell_batch)
        Q_targets_next_max, __ = Q_targets_next.detach().max(dim=1)
        print(Q_targets_next_max.shape) # torch.Size([23328, 1]) -> torch.Size([32])이 나와야함. -> 완료

        # q_value_target을 계산한다.
        Q_targets = reward + (gamma * Q_targets_next_max * (1 - dones))

        # main_network를 통해 현재 state와 action 대한 q_value를 도출한다.
        Q_expected, _ = self.main_network.forward(states, bsize=self.batch_size, time_step=1, hidden_state=hidden_batch, cell_state=cell_batch)
        # print(Q_expected.shape) # (32,729)
        Q_expected_action = Q_expected.gather(dim=1, index = actions)
        # print(Q_expected_action.shape) # (32,1)


        loss = F.mse_loss(Q_expected_action, Q_targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


In [162]:
class ReplayBuffer:

    def __init__(self, action_size, buffer_size, batch_size, seed):

        self.action_size = action_size #각각의 action의 차원
        self.memory = deque(maxlen=buffer_size)  #버퍼의 최대 크기
        self.batch_size = batch_size # 배치 사이즈
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed) # 랜덤 시드
    
    # states, actions, rewards, next_states, done을 replay memory에 저장하는 함수
    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    # replay memory에서 이전 과정에서 저장된 states, actions, rewards, next_states, done을 랜덤 추출하는 함수
    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        # print(states.shape) # (23328,6)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        # print(actions.shape) # (32,1)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        # print(rewards.shape) # (32,1)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)

In [163]:
# gru도 비교해보면 좋다.

class Network(nn.Module):
    
    def __init__(self,input_size,out_size):
        super(Network,self).__init__()
        self.input_size = input_size
        self.out_size = out_size
        
        self.conv_layer1 = nn.Conv1d(in_channels=1,out_channels=32,kernel_size=8,stride=4)
        self.conv_layer2 = nn.Conv1d(in_channels=32,out_channels=64,kernel_size=4,stride=2)
        self.conv_layer3 = nn.Conv1d(in_channels=64,out_channels=64,kernel_size=3,stride=1)
        self.conv_layer4 = nn.Conv1d(in_channels=64,out_channels=512,kernel_size=7,stride=1)
        self.lstm_layer = nn.LSTM(input_size=512,hidden_size=512,num_layers=1,batch_first=True)
        self.adv = nn.Linear(in_features=512,out_features=self.out_size)
        self.val = nn.Linear(in_features=512,out_features=1)
        self.relu = nn.ReLU()
        
    def forward(self,x,bsize,time_step,hidden_state,cell_state):
        x = x.view(bsize*time_step,1,-1)
        
        conv_out = self.conv_layer1(x)
        conv_out = self.relu(conv_out)
        conv_out = self.conv_layer2(conv_out)
        conv_out = self.relu(conv_out)
        conv_out = self.conv_layer3(conv_out)
        conv_out = self.relu(conv_out)
        conv_out = self.conv_layer4(conv_out)
        conv_out = self.relu(conv_out)
        # print(conv_out.shape) # [1,512,~]
        
        conv_out = conv_out.view(bsize, -1, 512)
        
        lstm_out = self.lstm_layer(conv_out,(hidden_state,cell_state))
        out = lstm_out[0][:,time_step-1,:]
        h_n = lstm_out[1][0]
        c_n = lstm_out[1][1]
        
        adv_out = self.adv(out)
        val_out = self.val(out)
        
        qout = val_out.expand(bsize,self.out_size) + (adv_out - adv_out.mean(dim=1).unsqueeze(dim=1).expand(bsize,self.out_size))
        
        return qout, (h_n,c_n)
    
    def init_hidden_states(self,bsize):
        h = torch.zeros(1,bsize,512).float().to(device)
        c = torch.zeros(1,bsize,512).float().to(device)
        
        return h,c

In [164]:
# simple network
q_network = nn.Sequential(
    nn.Linear(6, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1))

In [165]:
class H2Y2:
    def __init__(self):
        self.agent = H2Y2_Agent()
        self.env = H2Y2_env()
        self.eps = 1.0 # 처음 epsilon
        
    def dqn(self, n_episodes=2000, max_t=1000, eps_end=0.01, eps_decay=0.995):
            
        for i_episode in range(1, n_episodes+1):
            state = self.env.reset()
            score = 0
            for t in range(max_t):
                action_index = self.agent.select_action(state, self.eps)
                next_state, reward, done = self.env.step(state, action_index)
                self.agent.step(state, action_index, reward, next_state, done)
                state = next_state
                score += reward
                if done:
                    break 
            self.scores_window.append(score)       # save most recent score
            self.scores.append(score)              # save most recent score
            self.eps = max(eps_end, eps_decay*self.eps) # decrease epsilon
            if np.mean(self.scores_window)>=200.0:
                break
        return np.mean(self.scores_window)

In [166]:
h2y2 = H2Y2()

h2y2.dqn()

torch.Size([32])


c:\Users\zkdlx\miniconda3\envs\h2y2\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


torch.Size([32])


c:\Users\zkdlx\miniconda3\envs\h2y2\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


torch.Size([32])


c:\Users\zkdlx\miniconda3\envs\h2y2\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


torch.Size([32])


c:\Users\zkdlx\miniconda3\envs\h2y2\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


KeyboardInterrupt: 

In [147]:
def main():
    env = H2Y2_env()
    agent = H2Y2_Agent()
    state = env.reset()
    print(len(state))
    next_state, reward, done = env.step(state, 48)
    print(reward)
    print(done)
    
    
if __name__=='__main__':
    main()
    
# 실험해보면 좋을 것
# 1. 다음달에 새로운 데이터가 추가되었을 떄도 잘 동작하는가 ( 데이터가 점진적으로 추가되었을 때 사용이 가능한가 )
# 2. 랜덤서치랑 베이지안 서치랑 비교

AttributeError: 'list' object has no attribute 'shape'